In [2]:
import umap
import numpy as np
import pandas as pd

import jscatter 

import matplotlib.pyplot as plt

import humap
from datasets import load_dataset

import time



In [3]:
# N = 200000 #600_000

# indices = np.random.choice(744469, N, replace=False).tolist()
# print("loading dataset")
# sentence_embeddings = np.load("gpt4all-j-prompts.npy")[indices]
# print(sentence_embeddings.shape)

In [4]:
# groovy = load_dataset('nomic-ai/gpt4all-j-prompt-generations', revision='v1.3-groovy')

# docs = groovy['train']['prompt']

# documents = []
# for i in range(len(indices)):
#     documents.append(docs[indices[i]])
# del docs



In [7]:
# datasets consisting of prompts
dataset = pd.read_csv('./../datasets/gpt4all-prompts.csv')
documents = dataset['prompt']

sentence_embeddings = np.load("./../datasets/gpt4all-prompts.npy")

N = sentence_embeddings.shape[0]
sentence_embeddings.shape

(1182073, 384)

In [ ]:
print("creating humap object")
hUmap = humap.HUMAP(np.array([0.2, 0.2]))

print("fitting hierarchy")
tic = time.time()
y = np.zeros(N)
hUmap.fit(sentence_embeddings, y)
toc = time.time()



creating humap object
fitting hierarchy


In [5]:
embedding2 = hUmap.transform(2)

done in 2.490076 seconds.
Computing similarity among landmarks... 
done in 4.372593 seconds.
Fitting the hierarchy level... 
done in 0.110083 seconds.
Associating data points to landmarks... 
done in 0.009720 seconds.
Storing information for the next hierarchy level... 
done in 0.575872 seconds.
Level construction: 7.643547


Freeing memory...
Hierarchy construction in 156.287444 seconds.

Initing low-dimensional representation... returning noise object
done in 0.006592 seconds.
Fixing term: 0.01
Embedding level 2 with 8000 data samples.


	completed 0 / 500 epochs
	completed 50 / 500 epochs
	completed 100 / 500 epochs
	completed 150 / 500 epochs
	completed 200 / 500 epochs
	completed 250 / 500 epochs
	completed 300 / 500 epochs
	completed 350 / 500 epochs
	completed 400 / 500 epochs
	completed 450 / 500 epochs
	completed 500 epochs

It took 3.71819 to embed.


In [41]:
hUmap.set_fixing_term(0.01)
hUmap.fix_datapoints(embedding2)
embedding1 = hUmap.transform(1)

Initing low-dimensional representation... returning noise object
done in 0.070173 seconds.
Fixing term: 0.01
Embedding level 1 with 148893 data samples.


	completed 0 / 500 epochs
	completed 50 / 500 epochs
	completed 100 / 500 epochs
	completed 150 / 500 epochs
	completed 200 / 500 epochs
	completed 250 / 500 epochs
	completed 300 / 500 epochs
	completed 350 / 500 epochs
	completed 400 / 500 epochs
	completed 450 / 500 epochs
	completed 500 epochs

It took 107.612 to embed.


In [42]:
hUmap.set_fixing_term(0.01)
hUmap.fix_datapoints(embedding1)
embedding = hUmap.transform(0)

Initing low-dimensional representation... returning noise object
done in 0.177783 seconds.
Fixing term: 0.01
Embedding level 0 with 744469 data samples.


	completed 0 / 500 epochs
	completed 50 / 500 epochs
	completed 100 / 500 epochs
	completed 150 / 500 epochs
	completed 200 / 500 epochs
	completed 250 / 500 epochs
	completed 300 / 500 epochs
	completed 350 / 500 epochs
	completed 400 / 500 epochs
	completed 450 / 500 epochs
	completed 500 epochs

It took 530.716 to embed.


In [43]:
original_indices2 = hUmap.original_indices(2)

In [44]:
original_indices1 = hUmap.original_indices(1)

In [45]:
documents2 = []
for i in range(len(original_indices2)):
    documents2.append(documents[original_indices2[i]])


In [46]:
documents1 = []
for i in range(len(original_indices1)):
    documents1.append(documents[original_indices1[i]])


In [47]:
import hdbscan
import seaborn as sns

from sklearn.cluster import KMeans

In [48]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, min_samples=15)
# clusterer = KMeans(n_clusters=10)
clusterer.fit(embedding)





HDBSCAN(min_cluster_size=100, min_samples=15)

In [58]:
palette = sns.color_palette('viridis', clusterer.labels_.max()).as_hex()

labels = clusterer.labels_+1

df = pd.DataFrame({    
    'id': np.arange(len(y)),
    'x': embedding[:, 0],
    'y': embedding[:, 1],
    'document': documents,
    'labels': labels,
    'original_indices': np.arange(len(y)),
})

df2 = pd.DataFrame({   
    'id': np.arange(len(embedding2)),
    'x': embedding2[:, 0],
    'y': embedding2[:, 1],
    'document': documents2,
    'labels': labels[hUmap.original_indices(2)],
    'original_indices': original_indices2,
})

df1 = pd.DataFrame({    
    'id': np.arange(len(embedding1)),
    'x': embedding1[:, 0],
    'y': embedding1[:, 1],
    'document': documents1,
    'labels': labels[hUmap.original_indices(1)],
    'original_indices': original_indices1,
})

config = dict(
    background_color='#FFFFFF',
#     color_by='labels',
#     color_map=['#999999'] + palette#['#FFFF00', '#1CE6FF', '#FF34FF', '#FF4A46', '#008941', '#006FA6', '#A30059', '#FFDBE5', '#7A4900', '#0000A6']
)

scatter = jscatter.Scatter(data=df, x='x', y='y', **config)
scatter1 = jscatter.Scatter(data=df1, x='x', y='y', **config)
scatter2 = jscatter.Scatter(data=df2, x='x', y='y', **config)

jscatter.link([scatter2, scatter1, scatter], rows=1, row_height=640, match_by='original_indices')

GridBox(children=(HBox(children=(VBox(children=(Button(button_style='primary', icon='arrows', layout=Layout(wi…

In [16]:
# clusterer = KMeans(n_clusters=15).fit(embedding)

In [21]:

labels

array([3, 5, 9, ..., 2, 7, 5], dtype=int32)

GridBox(children=(HBox(children=(VBox(children=(Button(button_style='primary', icon='arrows', layout=Layout(wi…

In [95]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))